In [1]:
import cv2
import numpy as np
import sys
import os
import math

print(cv2.__version__)

3.4.3


### Inference config

In [2]:
IMG_NAME = "butterfly_GT.bmp"
#IMG_NAME = "lenna.bmp"

IMG_OPENCV_NAME = "opencv.png"
IMG_DNN_NAME = "srcnn.png"

INTERPOLATION = cv2.INTER_CUBIC
SCALE = 2
SIZE_CONV = 6

PSNR = True

### OpenCV dnn config

In [3]:
net = cv2.dnn.readNetFromTensorflow('yayoi_srcnn_935_2x.pb')

# computation backends
# "0: default C++ backend
# "1: Halide language (http://halide-lang.org/)
# "2: Intel's Deep Learning Inference Engine (https://software.seek.intel.com/deep-learning-deployment)}
net.setPreferableBackend(0)

# target computation devices
# "0: CPU target (by default)
# "1: OpenCL
net.setPreferableTarget(0)

### Generate OpenCV resized image for reference

In [4]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
shape = img.shape

if PSNR:
    img = cv2.resize(img, (int(shape[1] / SCALE), int(shape[0] / SCALE)), INTERPOLATION)
    img = cv2.resize(img, (shape[1], shape[0]), INTERPOLATION)
else:
    img = cv2.resize(img, (shape[1] * SCALE, shape[0] * SCALE), INTERPOLATION)

cv2.imwrite(IMG_OPENCV_NAME, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
tick1 = cv2.getTickCount()

### Generate super resolution image by SRCNN

In [ ]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
shape = img.shape

if PSNR:
    imgY = cv2.resize(img[:, :, 0], (int(shape[1] / SCALE), int(shape[0] / SCALE)), INTERPOLATION)
    imgY = cv2.resize(imgY, (shape[1], shape[0]), cv2.INTER_CUBIC)
    img[:, :, 0] = imgY
else:
    imgY = cv2.resize(img[:, :, 0], (int(shape[1] * SCALE), int(shape[0] * SCALE)), INTERPOLATION)
    img[:, :, 0] = imgY



### PSNR calculation

In [ ]:
im1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
im2 = cv2.imread(IMG_OPENCV_NAME, cv2.IMREAD_COLOR)
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
im3 = cv2.imread(IMG_DNN_NAME, cv2.IMREAD_COLOR)
im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)

if PSNR:
    print("opencv:")
    print(cv2.PSNR(im1, im2))
    print("srcnn:")
    print(cv2.PSNR(im1, im3))

### Display image

In [ ]:
pltShow = False

if pltShow:    
    from matplotlib import pyplot as plt
    %matplotlib inline
    import seaborn as sns
    
    plt.figure(num='comparison',figsize=(16,16))

    plt.subplot(1,3,1)
    plt.title('origin image')
    plt.imshow(im1)

    plt.subplot(1,3,2)
    plt.title('OpenCV')
    plt.imshow(im2)

    plt.subplot(1,3,3)
    plt.title("srcnn")
    plt.imshow(im3)

In [ ]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Inference processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Inference processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Inference processed time: " + str(tick) + " ms")